In [2]:
import pandas as pd
d =pd.read_csv("ccdefault.csv")
from sklearn.preprocessing import LabelEncoder
X = d.iloc[:,1:23]
y = d.iloc[:,-1]
X.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5
0,20000,2,2,1,24,2,2,-1,-1,-2,...,3102,689,0,0,0,0,689,0,0,0
1,120000,2,2,2,26,-1,2,0,0,0,...,1725,2682,3272,3455,3261,0,1000,1000,1000,0
2,90000,2,2,2,34,0,0,0,0,0,...,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000
3,50000,2,2,1,37,0,0,0,0,0,...,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069
4,50000,1,2,1,57,-1,0,-1,0,0,...,5670,35835,20940,19146,19131,2000,36681,10000,9000,689


# 1. Decision Tree

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import timeit
#where we use it to count down 
start = timeit.default_timer()
train_acc = []
test_acc =[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=i)
    tree = DecisionTreeClassifier()
    tree.fit(X_train, y_train)
    y_pred_test=tree.predict(X_test)
    y_pred_train=tree.predict(X_train)
    train_acc.append(round(accuracy_score(y_train,y_pred_train),5))
    test_acc.append(round(accuracy_score(y_test,y_pred_test),5)) 
stop = timeit.default_timer()

from prettytable import PrettyTable
#PrettyTable is not something that Anaconda has as default, which you had to
# download through terminal by "conda install -c conda-forge prettytable"
#might not be worthy of the time wasting on a program creating table
# but at least it looks good
output_train_acc = ["Training results",round(np.mean(train_acc),5),round(np.std(train_acc),5)]
output_test_acc =["Testing results",round(np.mean(test_acc),5),round(np.std(test_acc),5)]
for i in range(10):
    output_train_acc.insert(-2,train_acc[i])
    output_test_acc.insert(-2,test_acc[i])

x = PrettyTable()
x.field_names = ["Samples size", "1","2","3","4","5","6","7","8","9","10","Mean","SD"]
x.add_row(output_train_acc)
x.add_row(output_test_acc)
print(x)
print('Time: ', round((stop - start),5))

+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|   Samples size   |    1    |    2    |    3    |    4    |    5    |    6    |    7    |    8    |    9    |    10   |   Mean  |    SD   |
+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Training results | 0.99937 |  0.9993 | 0.99933 | 0.99933 |  0.9993 | 0.99933 | 0.99937 |  0.9993 | 0.99922 | 0.99948 | 0.99933 |  6e-05  |
| Testing results  |  0.726  | 0.71733 | 0.74067 |  0.729  | 0.71033 | 0.73233 | 0.71233 | 0.72533 | 0.72233 |   0.73  | 0.72457 | 0.00884 |
+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
Time:  5.56674


# 2. Cross Validation

In [4]:
from sklearn.model_selection import cross_val_score
#where we use it to count down 
start = timeit.default_timer()
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred_test=tree.predict(X_test)
y_pred_train=tree.predict(X_train)
train_acc = np.round(cross_val_score(tree,X=X_train,y=y_train,cv=10,n_jobs=-1),5)
test_acc = np.round(cross_val_score(tree,X=X_test,y=y_test,cv=10,n_jobs=-1),5)
stop = timeit.default_timer()

from prettytable import PrettyTable
output_train_acc= ["Training results",round(np.mean(train_acc),5),round(np.std(train_acc),5)]
output_test_acc=["Testing results",round(np.mean(test_acc),5),round(np.std(test_acc),5)]
for i in range(10):
    output_train_acc.insert(-2,train_acc[i])
    output_test_acc.insert(-2,test_acc[i])

x = PrettyTable()
x.field_names = ["Samples size", "1","2","3","4","5","6","7","8","9","10","Mean","SD"]
x.add_row(output_train_acc)
x.add_row(output_test_acc)
print(x)
print('Time: ', round((stop - start),5)) 

+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|   Samples size   |    1    |    2    |    3    |    4    |    5    |    6    |    7    |    8    |    9    |    10   |   Mean  |    SD   |
+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Training results | 0.72603 | 0.71455 | 0.70926 |   0.72  | 0.72556 | 0.72593 | 0.71815 | 0.72556 | 0.73138 | 0.71953 |  0.7216 | 0.00621 |
| Testing results  | 0.66777 | 0.68771 | 0.74333 | 0.69667 |   0.71  | 0.72667 | 0.72667 |   0.69  | 0.71237 | 0.72241 | 0.70836 | 0.02158 |
+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
Time:  6.58971


In [5]:
print("My name is {Qianyi Liu}")
print("My NetID is: {qianyil2}")
print("I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.")

My name is {Qianyi Liu}
My NetID is: {qianyil2}
I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.
